### [1] 전기차충전소 분포 현황 <hr>

모듈 로딩) <hr>

In [1]:
# 모듈 로딩
import pandas as pd
import matplotlib.pyplot as plt
import os
import folium

from folium import Marker
from folium.plugins import MarkerCluster # 클러스터를 표현하기 위한 모듈

한글 폰트 설정) <hr>

In [2]:
# ============================================================================
# 한글 폰트 설정 - 간단한 방법
# ============================================================================
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print(f"현재 폰트: {plt.rcParams['font.family']}\n")

현재 폰트: ['Malgun Gothic']



데이터 로드) <hr>

In [3]:
# ============================================================================
# 필요한 데이터 로드
# ============================================================================

# 데이터 로드
charger_location = '../data/raw/charger_location.csv'

df_charger = pd.read_csv(charger_location, encoding='cp949', low_memory=False)

print("=" * 80)
print("전기차 충전소 데이터 분석")
print("=" * 80)

print(f"\n총 데이터: {len(df_charger)}개 행")
print(f"컬럼: {list(df_charger.columns)}")
display(df_charger.head(1))

전기차 충전소 데이터 분석

총 데이터: 361163개 행
컬럼: ['설치년도', '시도', '군구', '주소', '충전소명', '시설구분(대)', '시설구분(소)', '기종(대)', '기종(소)', '운영기관(대)', '운영기관(소)', '급속충전량', '충전기타입', '이용자제한', '위도경도']


,설치년도,시도,군구,주소,충전소명,시설구분(대),시설구분(소),기종(대),기종(소),운영기관(대),운영기관(소),급속충전량,충전기타입,이용자제한,위도경도
0,2017,서울특별시,강서구,서울특별시 강서구 가로공원로 189,가로공원로 지하공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5372628,126.8383789"


데이터 전처리) <hr>

In [ ]:
# ============================================================================
# 데이터 전처리(필요없는 열 삭제)(['설치년도', '시도']만 남겨둠)
# ============================================================================
df_charger = df_charger[['설치년도', '시도']]
display(df_charger)

# ============================================================================
# 저장 경로 설정
# ============================================================================
save_path = '../data/processed'
os.makedirs(save_path, exist_ok=True)   # 폴더 없으면 자동 생성

# ============================================================================
# 원본 전처리 데이터 저장
# ============================================================================
csv_file = os.path.join(save_path, 'charger_preprocessed.csv')
df_charger.to_csv(csv_file, index=False, encoding='utf-8-sig')
print(f"\n✓ 전처리된 데이터 저장: {os.path.abspath(csv_file)}")

,설치년도,시도,위도경도
0,2017,서울특별시,"37.5372628,126.8383789"
1,2017,서울특별시,"37.5289561,126.8490887"
2,2017,서울특별시,"37.5419747,126.8444499"
3,2017,서울특별시,"37.5530457,126.8488169"
4,2017,서울특별시,"37.4874046,126.9235075"
...,...,...,...
361158,2015,광주광역시,NaN
361159,2016,전라남도,NaN
361160,2016,전라남도,NaN
361161,2011,전라남도,NaN



✓ 전처리된 데이터 저장: c:\Users\khw27\바탕 화면\MINI2 Pandas\data\processed\charger_preprocessed.csv


시도별 충전소 분포 분석) <hr>

In [5]:
# ============================================================================
# 시도별 충전소 분포 분석
# ============================================================================
print("\n" + "=" * 80)
print("시도별 충전소 분포")
print("=" * 80)

total_chargers = len(df_charger)                                                # 전체 충전소 수

region_stats = df_charger['시도'].value_counts().sort_values(ascending=False)    # 오름차순 정렬

for region, count in region_stats.items():                                      # 시도(key), 개수(value)  
    percentage = (count / total_chargers) * 100                                 # 지역별 개수 / 전체 충전소 * 100
    print(f"  - {region}: {count}개 ({percentage:.1f}%)")

# ============================================================================
# 시도별 충전소 개수를 새로운 파일에 저장
# ============================================================================
# 시도별 통계 데이터프레임 생성
region_stats_df = pd.DataFrame({'시도': region_stats.index,
                                '충전소_개수': region_stats.values,
                                '비율_%': (region_stats.values / total_chargers * 100).round(1)})

# 새로운 CSV 파일로 저장
save_path = '../reports/tables'
stats_csv_file = os.path.join(save_path, 'region_stats.csv')
region_stats_df.to_csv(stats_csv_file, index=False, encoding='utf-8-sig')
print(f"\n✓ 시도별 충전소 개수 저장: {os.path.abspath(stats_csv_file)}")


시도별 충전소 분포
  - 경기도: 98048개 (27.1%)
  - 서울특별시: 57088개 (15.8%)
  - 부산광역시: 21749개 (6.0%)
  - 경상남도: 19798개 (5.5%)
  - 인천광역시: 19495개 (5.4%)
  - 대구광역시: 18332개 (5.1%)
  - 경상북도: 17084개 (4.7%)
  - 충청남도: 15797개 (4.4%)
  - 강원특별자치도: 12893개 (3.6%)
  - 전북특별자치도: 12626개 (3.5%)
  - 전라남도: 12192개 (3.4%)
  - 충청북도: 12042개 (3.3%)
  - 제주특별자치도: 11729개 (3.2%)
  - 대전광역시: 10990개 (3.0%)
  - 광주광역시: 10249개 (2.8%)
  - 울산광역시: 6412개 (1.8%)
  - 세종특별자치시: 4639개 (1.3%)

✓ 시도별 충전소 개수 저장: c:\Users\khw27\바탕 화면\MINI2 Pandas\reports\tables\region_stats.csv


연도별/누적 설치 현황 분석) <hr>

In [6]:
# ============================================================================
# 연도별 설치 현황 분석
# ============================================================================
print("\n" + "=" * 80)
print("연도별 충전소 설치 현황")
print("=" * 80)

if '설치년도' in df_charger.columns:

    yearly_stats = df_charger['설치년도'].value_counts().sort_index()

    print(f"연도별 설치 현황")

    for year, count in yearly_stats.items():                            # 설치년도(key), 개수(values)
        percentage = (count / total_chargers) * 100                     # 연도별 개수 / 전체 충전소 * 100
        print(f"  - {year}년: {count}개 ({percentage:.1f}%)")

    # ============================================================================
    # 연도별 설치 현황을 새로운 파일에 저장
    # ============================================================================
    yearly_stats.df = pd.DataFrame({'연도': yearly_stats.index,
                                    '설치 개수': yearly_stats.values,
                                    '비율_%': (yearly_stats.values / total_chargers * 100).round(1)})
    
    # 새로운 CSV 파일로 저장
    save_path = '../reports/tables'
    stats_csv_file = os.path.join(save_path, 'yearly_stats.csv')
    yearly_stats.df.to_csv(stats_csv_file, index=False, encoding='utf-8-sig')
    print(f"\n✓ 연도별 충전소 설치 현황 저장: {os.path.abspath(stats_csv_file)}")

    # ============================================================================
    # 누적 설치 현황 분석
    # ============================================================================
    cumulative_stats = yearly_stats.cumsum()      # cumsum(): 누적합을 계산하는 pandas 메서드
    print(f"\n누적 설치 현황:")

    for year, cum_count in cumulative_stats.items():
        print(f"  - {year}년: {cum_count}개")
    
    # ============================================================================
    # 누적 설치 현황을 새로운 파일에 저장
    # ============================================================================
    cumulative_stats_df = pd.DataFrame({'연도': cumulative_stats.index,
                                  '설치 개수': cumulative_stats.values})
    
    # 새로운 CSV 파일로 저장
    save_path = '../reports/tables'
    stats_csv_file = os.path.join(save_path, 'cumulative_stats.csv')
    cumulative_stats_df.to_csv(stats_csv_file, index=False, encoding='utf-8-sig')
    print(f"\n✓ 연도별 충전소 누적 설치 현황 저장: {os.path.abspath(stats_csv_file)}")


연도별 충전소 설치 현황
연도별 설치 현황
  - 2011년: 7개 (0.0%)
  - 2012년: 19개 (0.0%)
  - 2013년: 37개 (0.0%)
  - 2014년: 60개 (0.0%)
  - 2015년: 176개 (0.0%)
  - 2016년: 697개 (0.2%)
  - 2017년: 9168개 (2.5%)
  - 2018년: 16768개 (4.6%)
  - 2019년: 21807개 (6.0%)
  - 2020년: 16962개 (4.7%)
  - 2021년: 34687개 (9.6%)
  - 2022년: 101316개 (28.1%)
  - 2023년: 110749개 (30.7%)
  - 2024년: 48710개 (13.5%)

✓ 연도별 충전소 설치 현황 저장: c:\Users\khw27\바탕 화면\MINI2 Pandas\reports\tables\yearly_stats.csv

누적 설치 현황:
  - 2011년: 7개
  - 2012년: 26개
  - 2013년: 63개
  - 2014년: 123개
  - 2015년: 299개
  - 2016년: 996개
  - 2017년: 10164개
  - 2018년: 26932개
  - 2019년: 48739개
  - 2020년: 65701개
  - 2021년: 100388개
  - 2022년: 201704개
  - 2023년: 312453개
  - 2024년: 361163개

✓ 연도별 충전소 누적 설치 현황 저장: c:\Users\khw27\바탕 화면\MINI2 Pandas\reports\tables\cumulative_stats.csv


변수 정리) <hr>
* 시도별 충전소 분포: region_stats

* 연도별 충전소 분포: yearly_stats

* 연도별 누적 충전소 분포: cumulative_stats

시각화 <hr>

In [7]:
# ============================================================================
# 시각화 (matplotlib)
# ============================================================================

print("\n" + "=" * 80)
print("시각화 생성")
print("=" * 80)


# 1. 시도별 충전소 분포 (막대그래프)
fig1, ax1 = plt.subplots(figsize=(8, 6))    # fig(그림판 전체), ax(그래프 영역)

region_stats = region_stats.sort_values(ascending=True)  # 내림차순 정렬
 
# barh(x축, y축, 막대 색상): 가로 막대 그래프 그리는 함수
bars = ax1.barh(region_stats.index, region_stats.values, color='skyblue')  

ax1.set_title('시도별 충전소 분포', fontsize=12, fontweight='bold')
ax1.set_xlabel('개수')

for bar in bars:                # bars: 각 막대 객체들의 리스트

    width = bar.get_width()     # 막대의 너비(width): 충전소의 개수

    # 막대에 텍스트 추가
    ax1.text(width, bar.get_y() + bar.get_height()/2,   # 텍스트의 위치: width(막대의 오른쪽 끝), 
                                                          #              bar.get_y(): 막대의 시작 y좌표(가로 막대 기준 막대의 아래쪽 위치)
                                                          #              bar.get_height(): 막대의 높이(굵기)
             f'{int(width):,}',     # 표시형식(천 단위 구분 콤마 포함)
             ha='left', va='center', fontsize=9, fontweight='bold') # 텍스트 정렬: 왼쪽, 수직 중앙, 글자크기, 글씨체

fig1.tight_layout() # 그래프의 요소들 겹치지 않게 자동 조정
fig1.savefig('../reports/figures/1_01_region_stats.png', dpi=300)
plt.close(fig1)     # 그래프 닫음(메모리 절약)
print("✓ 그래프 저장: reports/figures/1_01_region_stats.png")


# 2. 시도별 충전소 비율 (막대그래프)
fig2, ax2 = plt.subplots(figsize=(8, 6))

total_count = region_stats.sum()
region_stats_percent = (region_stats / total_count * 100).sort_values(ascending=True)

bars2 = ax2.barh(region_stats_percent.index, region_stats_percent.values, color='coral')

ax2.set_title('시도별 충전소 비율', fontsize=12, fontweight='bold')
ax2.set_xlabel('비율 (%)')

for bar in bars2:

    width = bar.get_width()

    ax2.text(width, bar.get_y() + bar.get_height()/2, 
             f'{width:.1f}%',
             ha='left', va='center', fontsize=9, fontweight='bold')

fig2.tight_layout()
fig2.savefig('../reports/figures/1_02_region_ratio.png', dpi=300)
plt.close(fig2)
print("✓ 그래프 저장: reports/figures/1_02_region_ratio.png")


# 3. 연도별 누적 충전소 분포 (라인그래프)
fig3, ax3 = plt.subplots(figsize=(8, 6))

ax3.plot(cumulative_stats.index, cumulative_stats.values,   # 인덱스(년도), 열(누적 개수)
            marker='s', color='green', linewidth=2, markersize=8)
#           사각형 마커 , 색상         , 선 굵기     , 마커 

ax3.set_title('누적 설치 현황', fontsize=12, fontweight='bold')
ax3.set_xlabel('년도')
ax3.set_ylabel('누적 개수')
ax3.grid(True, alpha=0.3)    # 배경 그리드 투명도를 설정해 가독성 높임

# 선에 텍스트 추가 
for x, y in zip(cumulative_stats.index, cumulative_stats.values):   # zin(x, y): x, y 하나씩 꺼내서 반복
    ax3.text(x, y, f'{int(y):,}', 
                ha='center', va='bottom', fontsize=9, fontweight='bold')
#               점의 가운데  , 텍스트 아래 쪽이 y값 바로 위   
              
fig3.tight_layout()
fig3.savefig('../reports/figures/1_03_cumulative_stats.png', dpi=300)
plt.close(fig3)
print("✓ 그래프 저장: reports/figures/1_03_cumulative_stats.png")


# 4. 연도별 충전소 분포 (라인그래프)
fig4, ax4 = plt.subplots(figsize=(8, 6))

ax4.plot(yearly_stats.index, yearly_stats.values, 
            marker='o', color='red', linewidth=2, markersize=8)

ax4.set_title('연도별 설치 현황', fontsize=12, fontweight='bold')
ax4.set_xlabel('년도')
ax4.set_ylabel('개수')
ax4.grid(True, alpha=0.3)

for x, y in zip(yearly_stats.index, yearly_stats.values):
    ax4.text(x, y, f'{int(y):,}', 
                ha='center', va='bottom', fontsize=9, fontweight='bold')

fig4.tight_layout()
fig4.savefig('../reports/figures/1_04_yearly_stats.png', dpi=300)
plt.close(fig4)
print("✓ 그래프 저장: reports/figures/1_04_yearly_stats.png")


시각화 생성
✓ 그래프 저장: reports/figures/1_01_region_stats.png
✓ 그래프 저장: reports/figures/1_02_region_ratio.png
✓ 그래프 저장: reports/figures/1_03_cumulative_stats.png
✓ 그래프 저장: reports/figures/1_04_yearly_stats.png
